In [5]:
from typing import List, Any, NamedTuple, Optional, Dict, TypeVar, Union
from collections import Counter, defaultdict
import math
import pprint

In [6]:
def entropy(class_probabilities: List[float]) -> float:
    """dada una loista de probabilidades de clase, calcula la entropia"""
    return sum(- p * math.log(p, 2) for p in class_probabilities if p > 0)

assert entropy([1.0]) == 0
assert entropy([0.5, 0.5]) == 1
assert 0.81 < entropy([0.25, 0.75]) < 0.82

In [7]:
def class_probabilities(labels: List[Any]) -> List[float]:
    total_count = len(labels)
    return [count / total_count for count in Counter(labels).values()]

def data_entropy(labels: List[Any]) -> float:
    return entropy(class_probabilities(labels))

assert data_entropy(['a']) == 0
assert data_entropy([True, False]) == 1
assert data_entropy([3, 4, 4, 4]) == entropy([0.25, 0.75])

In [8]:
def partition_entropy(subsets: List[List[Any]]) -> float:
    """devuelve la entropia de esta particion de datos en subconjuntos"""
    total_count = sum(len(subset) for subset in subsets)
    return sum(data_entropy(subset) * len(subset) / total_count for subset in subsets)

In [9]:
class Candidate(NamedTuple):
    level: str
    lang: str
    tweets: bool
    phd: bool
    did_well: Optional[bool] = None  
           # permite datos sin etiquetar
inputs = [Candidate('Senior', 'Java',   False, False, False),Candidate('Senior', 'Java',   False, True,  False),Candidate('Mid',    'Python', False, False, True),Candidate('Junior', 'Python', False, False, True),Candidate('Junior', 'R',      True,  False, True),Candidate('Junior', 'R',      True,  True,  False),Candidate('Mid',    'R',      True,  True,  True),Candidate('Senior', 'Python', False, False, False),Candidate('Senior', 'R',      True,  False, True),Candidate('Junior', 'Python', True,  False, True),Candidate('Senior', 'Python', True,  True,  True),Candidate('Mid',    'Python', False, True,  True),Candidate('Mid',    'Java',   True,  False, True),Candidate('Junior', 'Python', False, True,  False)]

In [10]:
T = TypeVar('T')
def partition_by(inputs: List[T], attribute: str) -> Dict[Any, List[T]]:
    """particionar las entradas en listas segun el atributomespecifico"""
    partitions: Dict[Any, List[T]] = defaultdict(list)
    for input in inputs:
        key = getattr(input, attribute)
        partitions[key].append(input)
    return partitions

In [11]:
def partitio_entropy_by(inputs: List[Any], attribute: str, label_attribute: str) -> float:
    """calcula la entropia correspondiente a la particion dada"""
    # partitions consiste en nuestras entradas
    partitions = partition_by(inputs, attribute)
    # pero partition_entropy solo necesita las etiquetas de clase
    labels = [[getattr(input, label_attribute) for input in partition] for partition in partitions.values()]
    return partition_entropy(labels)

In [12]:
for key in ['level', 'lang', 'tweets', 'phd']:
    print(key, partitio_entropy_by(inputs, key, 'did_well'))

level 0.6935361388961919
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617


In [13]:
senios_inputs = [input for input in inputs if input.level == 'Senior']
senios_inputs

[Candidate(level='Senior', lang='Java', tweets=False, phd=False, did_well=False),
 Candidate(level='Senior', lang='Java', tweets=False, phd=True, did_well=False),
 Candidate(level='Senior', lang='Python', tweets=False, phd=False, did_well=False),
 Candidate(level='Senior', lang='R', tweets=True, phd=False, did_well=True),
 Candidate(level='Senior', lang='Python', tweets=True, phd=True, did_well=True)]

In [14]:
class Leaf(NamedTuple):
    value: Any

class Split(NamedTuple):
    attribute: str
    subtrees: dict
    deafault_value: Any = None

DecisionTree = Union[Leaf, Split]

In [15]:
hiring_tree = Split('level', {
    'Junior': Split('tweets', {
        False: Leaf(True),
        True: Leaf(False)
    }),
    'Mid': Leaf(True),
    'Senior': Split('tweets', {
        False: Leaf(False),
        True: Leaf(True)
    })
})

In [16]:
def classify(tree: DecisionTree, input: Any) -> Any:
    """clasifica una entrada usando el arbol de decision dado"""
    # si es un node de hoja, devuelve su valor
    if isinstance(tree, Leaf):
        return tree.value
    # si no  este arbol consiste en un atributo segun el que dividir y ub diccionario cuyas claves son valores de ese atributo y cuyos valores son subarboles que considerar despues
    subtree_key = getattr(input, tree.attribute)
    if subtree_key not in tree.subtrees:            # si no hay subarbol para la clave
        return tree.default_value                   # devuelve el valor por defecto
    subtree = tree.subtrees[subtree_key]            # elige el subarbol adecuado
    return classify(subtree, input)                 # y lo usa para clasificar la entrada

In [17]:
def build_tree_id3(inputs: List[Any], split_attributes: List[str], target_attribute: str) -> DecisionTree:
    # cuenta las etiquetas de clase
    label_counts = Counter(getattr(input, target_attribute) for input in inputs)
    most_common_label = label_counts.most_common(1)[0][0]
    # si solo hay una etiqueta de clase, devuelve una hoja con esa etiqueta
    if len(label_counts) == 1:
        return Leaf(most_common_label)
    # si no hay atributos para dividir, devuelve una hoja con la etiqueta mas comun
    if not split_attributes:
        return Leaf(most_common_label)
    # de lo contrario, divide por el mejor atributo
    
    def split_entropy(attribute: str) -> float:
        return partitio_entropy_by(inputs, attribute, target_attribute)
    best_attribute = min(split_attributes, key=split_entropy)
    partitions = partition_by(inputs, best_attribute)
    new_attributes = [a for a in split_attributes if a != best_attribute]
    subtrees = {attribute_value: build_tree_id3(subset, new_attributes, target_attribute) for attribute_value, subset in partitions.items()}
    return Split(best_attribute, subtrees, most_common_label)

In [20]:
tree = build_tree_id3(inputs, ['level', 'lang', 'tweets', 'phd'], 'did_well')

pprint.pprint(tree)

Split(attribute='level', subtrees={'Senior': Split(attribute='tweets', subtrees={False: Leaf(value=False), True: Leaf(value=True)}, deafault_value=False), 'Mid': Leaf(value=True), 'Junior': Split(attribute='phd', subtrees={False: Leaf(value=True), True: Leaf(value=False)}, deafault_value=True)}, deafault_value=True)


In [ ]:
# si ya hay suficicentes candidatos divididos, los mira todos
if len(split_candidates) <= self.num_split_candidate:
    sampled_splir_candidates = split_candidates
# si no elige una muestra alatoria
else:
    sampled_split_candidates = random.sample(split_candidates, self.num_split_candidates)